# 0. Import modules

In [340]:
import gc
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

from pathlib import Path

# 1.Load data

In [341]:
### Paths ###
DATA_PATH = Path("../data")
TRAIN_RAW_PATH = DATA_PATH / "raw/train.csv"
TEST_RAW_PATH = DATA_PATH / "raw/test.csv"
SAMPLE_SUBMISSION_RAW_PATH = DATA_PATH / "raw/sample_submission.csv"

DATA_PROCESSED_PATH = DATA_PATH / "processed"

In [342]:
census = pd.read_csv(DATA_PATH / "raw/census_starter.csv")
train = pd.read_csv(DATA_PATH / "raw/train.csv")
reaveal_test = pd.read_csv(DATA_PATH / "raw/revealed_test.csv")
test = pd.read_csv(DATA_PATH / "raw/test.csv")
sub = pd.read_csv(DATA_PATH / "raw/sample_submission.csv")
coords = pd.read_csv(DATA_PATH / "external/cfips_location.csv")

In [343]:
test.head()

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01


In [344]:
train = (
    pd.concat([train, reaveal_test])
    .sort_values(by=["cfips", "first_day_of_month"])
    .reset_index()
)

drop_index = (test.first_day_of_month == "2022-11-01") | (
    test.first_day_of_month == "2022-12-01"
)
test = test.loc[~drop_index, :]

train["istest"] = 0
test["istest"] = 1
raw = pd.concat((train, test)).sort_values(["cfips", "row_id"]).reset_index(drop=True)
raw = raw.merge(coords.drop("name", axis=1), on="cfips")

raw["state_i1"] = raw["state"].astype("category")
raw["county_i1"] = raw["county"].astype("category")
raw["first_day_of_month"] = pd.to_datetime(raw["first_day_of_month"])
raw["county"] = raw.groupby("cfips")["county"].ffill()
raw["state"] = raw.groupby("cfips")["state"].ffill()
raw["dcount"] = raw.groupby(["cfips"])["row_id"].cumcount()
raw["county_i"] = (raw["county"] + raw["state"]).factorize()[0]
raw["state_i"] = raw["state"].factorize()[0]
raw["scale"] = (raw["first_day_of_month"] - raw["first_day_of_month"].min()).dt.days
raw["scale"] = raw["scale"].factorize()[0]

In [345]:
raw.head()

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,lat,state_i1,county_i1,dcount,county_i,state_i,scale
0,0.0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,0,-86.6429,32.535142,Alabama,Autauga County,0,0,0,0
1,1.0,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,-86.6429,32.535142,Alabama,Autauga County,1,0,0,1
2,2.0,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,-86.6429,32.535142,Alabama,Autauga County,2,0,0,2
3,3.0,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,-86.6429,32.535142,Alabama,Autauga County,3,0,0,3
4,4.0,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,-86.6429,32.535142,Alabama,Autauga County,4,0,0,4


# 2.Anomaly Detection

In [346]:
# There are some anomalies, specially at timestep 18
for o in tqdm(raw.cfips.unique()):
    indices = raw["cfips"] == o
    tmp = raw.loc[indices].copy().reset_index(drop=True)
    var = tmp.microbusiness_density.values.copy()
    for i in range(37, 2, -1):
        thr = 0.10 * np.mean(var[:i])
        difa = var[i] - var[i - 1]
        if (difa >= thr) or (difa <= -thr):
            if difa > 0:
                var[:i] += difa - 0.003
            else:
                var[:i] += difa + 0.003
    var[0] = var[1] * 0.99
    raw.loc[indices, "microbusiness_density"] = var

  0%|          | 0/3135 [00:00<?, ?it/s]

In [347]:
lag = 1
raw[f"mbd_lag_{lag}"] = raw.groupby("cfips")["microbusiness_density"].shift(lag).bfill()
raw["dif"] = (raw["microbusiness_density"] / raw[f"mbd_lag_{lag}"]).fillna(1).clip(
    0, None
) - 1
raw.loc[(raw[f"mbd_lag_{lag}"] == 0), "dif"] = 0
raw.loc[(raw[f"microbusiness_density"] > 0) & (raw[f"mbd_lag_{lag}"] == 0), "dif"] = 1
raw["dif"] = raw["dif"].abs()
# raw.groupby('dcount')['dif'].sum().plot()

# 3.Feature Engineering

In [348]:
def build_features(
    raw, target="microbusiness_density", target_act="active_tmp", lags=6
):
    feats = []

    for lag in range(1, lags):
        raw[f"mbd_lag_{lag}"] = raw.groupby("cfips")[target].shift(lag)
        raw[f"act_lag_{lag}"] = raw.groupby("cfips")[target_act].diff(lag)
        feats.append(f"mbd_lag_{lag}")
        feats.append(f"act_lag_{lag}")

    lag = 1
    for window in [2, 4, 6, 8, 10]:
        raw[f"mbd_rollmea{window}_{lag}"] = raw.groupby("cfips")[
            f"mbd_lag_{lag}"
        ].transform(lambda s: s.rolling(window, min_periods=1).sum())
        feats.append(f"mbd_rollmea{window}_{lag}")

    census_columns = list(census.columns)
    census_columns.remove("cfips")

    raw = raw.merge(census, on="cfips", how="left")
    feats += census_columns

    co_est = pd.read_csv(DATA_PATH / "external/co-est2021-alldata.csv", encoding="latin-1")
    
    co_est["cfips"] = co_est.STATE * 1000 + co_est.COUNTY
    co_columns = [
        "SUMLEV",
        "DIVISION",
        "ESTIMATESBASE2020",
        "POPESTIMATE2020",
        "POPESTIMATE2021",
        "NPOPCHG2020",
        "NPOPCHG2021",
        "BIRTHS2020",
        "BIRTHS2021",
        "DEATHS2020",
        "DEATHS2021",
        "NATURALCHG2020",
        "NATURALCHG2021",
        "INTERNATIONALMIG2020",
        "INTERNATIONALMIG2021",
        "DOMESTICMIG2020",
        "DOMESTICMIG2021",
        "NETMIG2020",
        "NETMIG2021",
        "RESIDUAL2020",
        "RESIDUAL2021",
        "GQESTIMATESBASE2020",
        "GQESTIMATES2020",
        "GQESTIMATES2021",
        "RBIRTH2021",
        "RDEATH2021",
        "RNATURALCHG2021",
        "RINTERNATIONALMIG2021",
        "RDOMESTICMIG2021",
        "RNETMIG2021",
    ]
    raw = raw.merge(co_est, on="cfips", how="left")
    feats += co_columns
    return raw, feats


def rot(df):
    for angle in [15, 30, 45]:
        df[f"rot_{angle}_x"] = (np.cos(np.radians(angle)) * df["lat"]) + (
            np.sin(np.radians(angle)) * df["lng"]
        )

        df[f"rot_{angle}_y"] = (np.cos(np.radians(angle)) * df["lat"]) - (
            np.sin(np.radians(angle)) * df["lng"]
        )

    return df

In [349]:
# SMAPE is a relative metric so target must be converted.
raw["target"] = raw.groupby("cfips")["microbusiness_density"].shift(-1)
raw["target"] = raw["target"] / raw["microbusiness_density"] - 1

raw.loc[raw["cfips"] == 28055, "target"] = 0.0
raw.loc[raw["cfips"] == 48269, "target"] = 0.0

In [350]:
raw["lastactive"] = raw.groupby("cfips")["active"].transform("last")

In [351]:
# Build Features based in lag of target
raw, feats = build_features(raw, "target", "active", lags=9)
features = ["state_i"]
features += feats
features += ["lng", "lat", "scale"]

In [352]:
# Latitude and Longitude feature engineering from samu2505.
coordinates = raw[["lng", "lat"]].values

# Encoding tricks
emb_size = 20
precision = 1e6

latlon = np.expand_dims(coordinates, axis=-1)

m = np.exp(np.log(precision) / emb_size)
angle_freq = m ** np.arange(emb_size)
angle_freq = angle_freq.reshape(1, 1, emb_size)
latlon = latlon * angle_freq
latlon[..., 0::2] = np.cos(latlon[..., 0::2])

In [353]:
raw = rot(raw)

In [354]:
features += ["rot_15_x", "rot_15_y", "rot_30_x", "rot_30_y", "rot_45_x", "rot_45_y"]

# 4.Model

In [355]:
# A bunch of useful functions

def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))

    num = np.abs(y_true - y_pred)
    dem = (np.abs(y_true) + np.abs(y_pred)) / 2

    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]

    return 100 * np.mean(smap)


def vsmape(y_true, y_pred):
    smap = np.zeros(len(y_true))

    num = np.abs(y_true - y_pred)
    dem = (np.abs(y_true) + np.abs(y_pred)) / 2

    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]

    return 100 * smap


def get_meta_model():
    from sklearn.ensemble import VotingRegressor
    import lightgbm as lgb
    import xgboost as xgb
    import catboost as cat
    from sklearn.pipeline import Pipeline
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.impute import KNNImputer

    # we should decrease the num_iterations of catboost
    cat_model = cat.CatBoostRegressor(
        iterations=2000,
        loss_function="MAPE",
        verbose=0,
        grow_policy="SymmetricTree",
        learning_rate=0.035,
        colsample_bylevel=0.8,
        max_depth=5,
        l2_leaf_reg=0.2,
        subsample=0.70,
        max_bin=4096,
    )

    return cat_model


def base_models():
    from sklearn.ensemble import VotingRegressor
    import lightgbm as lgb
    import xgboost as xgb
    import catboost as cat
    from sklearn.pipeline import Pipeline
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.impute import KNNImputer

    # LGBM model
    params = {
        "n_iter": 300,
        "boosting_type": "dart",
        "verbosity": -1,
        "objective": "l1",
        "random_state": 42,
        "colsample_bytree": 0.8841279649367693,
        "colsample_bynode": 0.10142964450634374,
        "max_depth": 8,
        "learning_rate": 0.003647749926797374,
        "lambda_l2": 0.5,
        "num_leaves": 61,
        "seed": 42,
        "min_data_in_leaf": 213,
    }

    lgb_model = lgb.LGBMRegressor(**params)

    xgb_model = xgb.XGBRegressor(
        objective="reg:pseudohubererror",
        tree_method="hist",
        n_estimators=795,
        learning_rate=0.0075,
        max_leaves=17,
        subsample=0.50,
        colsample_bytree=0.50,
        max_bin=4096,
        n_jobs=2,
    )

    # we should decrease the num_iterations of catboost
    cat_model = cat.CatBoostRegressor(
        iterations=2500,
        loss_function="MAPE",
        verbose=0,
        grow_policy="SymmetricTree",
        learning_rate=0.035,
        colsample_bylevel=0.8,
        max_depth=5,
        l2_leaf_reg=0.2,
        subsample=0.70,
        max_bin=4096,
    )

    models = {}
    models["xgb"] = xgb_model
    models["lgbm"] = lgb_model
    models["cat"] = cat_model

    return models


def from_active_to_density(submission):
    COLS = ["GEO_ID", "NAME", "S0101_C01_026E"]
    df2020 = pd.read_csv(
        DATA_PATH / "external/census/ACSST5Y2020.S0101-Data.csv", usecols=COLS
    )
    df2020 = df2020.iloc[1:]
    df2020["S0101_C01_026E"] = df2020["S0101_C01_026E"].astype("int")

    df2021 = pd.read_csv(
        DATA_PATH / "external/census/ACSST5Y2021.S0101-Data.csv", usecols=COLS
    )
    df2021 = df2021.iloc[1:]
    df2021["S0101_C01_026E"] = df2021["S0101_C01_026E"].astype("int")
    df2021.head()

    df2020["cfips"] = df2020.GEO_ID.apply(lambda x: int(x.split("US")[-1]))
    adult2020 = df2020.set_index("cfips").S0101_C01_026E.to_dict()

    df2021["cfips"] = df2021.GEO_ID.apply(lambda x: int(x.split("US")[-1]))
    adult2021 = df2021.set_index("cfips").S0101_C01_026E.to_dict()

    submission["adult2020"] = submission.cfips.map(adult2020)
    submission["adult2021"] = submission.cfips.map(adult2021)

    submission["microbusiness_density"] = (
        submission["microbusiness_density"].round() * 100 / submission["adult2021"]
    )

In [356]:
ACT_THR = 150
MONTH_1 = 39
MONTH_last = 40

In [357]:
raw[raw.istest == 0].shape


(128535, 107)

In [358]:
raw["ypred_last"] = np.nan
raw["ypred"] = np.nan
raw["k"] = 1.0
raw["microbusiness_density"].fillna(0, inplace=True)


for TS in range(MONTH_1, MONTH_last):  # 40):
    print(f"TS: {TS}")

    models = base_models()
    model0 = models["xgb"]
    model1 = models["lgbm"]
    model2 = models["cat"]

    train_indices = (
        (raw.istest == 0)
        & (raw.dcount < TS)
        & (raw.dcount >= 1)
        & (raw.lastactive > ACT_THR)
    )
    valid_indices = (raw.istest == 0) & (raw.dcount == TS)

    # Train each of the models on the current TS
    model0.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, "target"].clip(-0.002, 0.006),
    )

    model1.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, "target"].clip(-0.002, 0.006),
    )

    model2.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, "target"].clip(-0.002, 0.006),
    )

    tr_pred0 = model0.predict(raw.loc[train_indices, features])
    tr_pred1 = model1.predict(raw.loc[train_indices, features])
    tr_pred2 = model2.predict(raw.loc[train_indices, features])
    train_preds = np.column_stack((tr_pred0, tr_pred1, tr_pred2))

    meta_model = get_meta_model()
    meta_model.fit(train_preds, raw.loc[train_indices, "target"].clip(-0.002, 0.006))

    val_preds0 = model0.predict(raw.loc[valid_indices, features])
    val_preds1 = model1.predict(raw.loc[valid_indices, features])
    val_preds2 = model2.predict(raw.loc[valid_indices, features])
    valid_preds = np.column_stack((val_preds0, val_preds1, val_preds2))

    ypred = meta_model.predict(valid_preds)
    raw.loc[valid_indices, "k"] = ypred + 1
    raw.loc[valid_indices, "k"] = (
        raw.loc[valid_indices, "k"] * raw.loc[valid_indices, "microbusiness_density"]
    )

    # Validate
    lastval = (
        raw.loc[raw.dcount == TS, ["cfips", "microbusiness_density"]]
        .set_index("cfips")
        .to_dict()["microbusiness_density"]
    )
    dt = raw.loc[raw.dcount == TS, ["cfips", "k"]].set_index("cfips").to_dict()["k"]

    df = raw.loc[
        raw.dcount == (TS + 1),
        ["cfips", "microbusiness_density", "state", "lastactive", "mbd_lag_1"],
    ].reset_index(drop=True)
    df["pred"] = df["cfips"].map(dt)
    df["lastval"] = df["cfips"].map(lastval)


    df.loc[df["lastactive"] <= ACT_THR, "pred"] = df.loc[
        df["lastactive"] <= ACT_THR, "lastval"
    ]

    raw.loc[raw.dcount == (TS + 1), "ypred"] = df["pred"].values
    raw.loc[raw.dcount == (TS + 1), "ypred_last"] = df["lastval"].values

    print("Last Value SMAPE:", smape(df["microbusiness_density"], df["lastval"]))
    print("SMAPE:", smape(df["microbusiness_density"], df["pred"]))
    print()

ind = (raw.dcount > MONTH_1) & (raw.dcount <= MONTH_last)

print("SMAPE:", smape(raw.loc[ind, "microbusiness_density"], raw.loc[ind, "ypred"]))
print(
    "Last Value SMAPE:",
    smape(raw.loc[ind, "microbusiness_density"], raw.loc[ind, "ypred_last"]),
)

TS: 39


/Users/Artem_Boltaev/opt/anaconda3/envs/lightgbm/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=300, n_iter=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
Last Value SMAPE: 1.889206717018118
SMAPE: 1.8637503450967807

SMAPE: 1.8637503450967807
Last Value SMAPE: 1.889206717018118


In [359]:
raw[raw["microbusiness_density"].isnull()]

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,RNETMIG2021,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k


In [360]:
TS = 40
print(TS)

model0 = get_meta_model()

train_indices = (
    (raw.istest == 0)
    & (raw.dcount < TS)
    & (raw.dcount >= 1)
    & (raw.lastactive > ACT_THR)
)
valid_indices = raw.dcount == TS

model0.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, "target"].clip(-0.002, 0.006),
)

model1.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, "target"].clip(-0.002, 0.006),
)

model2.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, "target"].clip(-0.002, 0.006),
)


tr_pred0 = model0.predict(raw.loc[train_indices, features])
tr_pred1 = model1.predict(raw.loc[train_indices, features])
tr_pred2 = model2.predict(raw.loc[train_indices, features])
train_preds = np.column_stack((tr_pred0, tr_pred1, tr_pred2))

meta_model = get_meta_model()
meta_model.fit(train_preds, raw.loc[train_indices, "target"].clip(-0.002, 0.006))

val_preds0 = model0.predict(raw.loc[valid_indices, features])
val_preds1 = model1.predict(raw.loc[valid_indices, features])
val_preds2 = model2.predict(raw.loc[valid_indices, features])
valid_preds = np.column_stack((val_preds0, val_preds1, val_preds2))

ypred = meta_model.predict(valid_preds)

# ypred = model0.predict(raw.loc[valid_indices, features])
raw.loc[valid_indices, "k"] = ypred + 1.0
raw.loc[valid_indices, "k"] = (
    raw.loc[valid_indices, "k"] * raw.loc[valid_indices, "microbusiness_density"]
)

# Validate
lastval = (
    raw.loc[raw.dcount == TS, ["cfips", "microbusiness_density"]]
    .set_index("cfips")
    .to_dict()["microbusiness_density"]
)
dt = raw.loc[raw.dcount == TS, ["cfips", "k"]].set_index("cfips").to_dict()["k"]

40


/Users/Artem_Boltaev/opt/anaconda3/envs/lightgbm/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=300, n_iter=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5


In [361]:
df = raw.loc[
    raw.dcount == (TS + 1),
    ["cfips", "microbusiness_density", "state", "lastactive", "mbd_lag_1"],
].reset_index(drop=True)

In [362]:
df["pred"] = df["cfips"].map(dt)
df["lastval"] = df["cfips"].map(lastval)

df.loc[df["lastactive"] <= ACT_THR, "pred"] = df.loc[
    df["lastactive"] <= ACT_THR, "lastval"
]

raw.loc[raw.dcount == (TS + 1), "ypred"] = df["pred"].values
raw.loc[raw.dcount == (TS + 1), "ypred_last"] = df["lastval"].values

In [363]:
mask = (raw.lastactive > 150) & (raw.dcount == (TS + 1))
k = 0.1
raw.loc[mask, "ypred"] = (
    k * 1.005 * raw.loc[mask, "ypred_last"] + (1 - k) * raw.loc[mask, "ypred"]
)

In [364]:
raw.loc[raw["cfips"] == 28055, "microbusiness_density"] = 0
raw.loc[raw["cfips"] == 48269, "microbusiness_density"] = 1.762115

In [365]:
raw.head()

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,RNETMIG2021,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k
0,0.0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,2.856021,1249.0,0,-86.6429,...,4.102668,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0
1,1.0,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,-86.6429,...,4.102668,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0
2,2.0,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,-86.6429,...,4.102668,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0
3,3.0,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,-86.6429,...,4.102668,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0
4,4.0,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,-86.6429,...,4.102668,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0


In [332]:
'''raw["cfips"] = raw["row_id"].map(lambda x: int(x.split("_")[0]))

# get adult_population
column_names = ["GEO_ID", "NAME", "S0101_C01_026E"]
df2021 = pd.read_csv(
    DATA_PATH / "external/census/ACSST5Y2021.S0101-Data.csv", usecols=column_names
)
df2021 = df2021.iloc[1:]
df2021["S0101_C01_026E"] = df2021["S0101_C01_026E"].astype("int")
df2021["cfips"] = df2021.GEO_ID.apply(lambda x: int(x.split("US")[-1]))
adult2021 = df2021.set_index("cfips").S0101_C01_026E.to_dict()

raw["adult2021"] = raw["cfips"].map(adult2021)

# rounding to the nearest integer

raw = raw.merge(df2021, on="cfips")

maska = raw["first_day_of_month"] == "2023-01-01"

raw.loc[maska, "microbusiness_density"] = (
    np.round(raw.loc[maska, "ypred"] * raw.loc[maska, "S0101_C01_026E_x"] / 100)
    / raw.loc[maska, "S0101_C01_026E_x"]
    * 100
)'''



COLS = ['GEO_ID','NAME','S0101_C01_026E']
df2020 = pd.read_csv( DATA_PATH / "external/census/ACSST5Y2020.S0101-Data.csv", usecols=COLS, dtype = 'object')
df2021 = pd.read_csv( DATA_PATH / "external/census/ACSST5Y2021.S0101-Data.csv",usecols=COLS, dtype = 'object')

df2020 = df2020.iloc[1:]
df2020 = df2020.astype({'S0101_C01_026E':'int'})

df2021 = df2021.iloc[1:]
df2021 = df2021.astype({'S0101_C01_026E':'int'})

df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

df2020['mnoshitel'] = df2020['S0101_C01_026E']/ df2021['S0101_C01_026E']

df2020 = df2020[['cfips','mnoshitel']]

In [333]:
raw.head()

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k,mnoshitel
0,0.0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,2.856021,1249.0,0,-86.6429,...,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0,42496
1,1.0,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,-86.6429,...,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0,42496
2,2.0,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,-86.6429,...,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0,42496
3,3.0,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,-86.6429,...,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0,42496
4,4.0,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,-86.6429,...,9.001702,53.851367,-15.14519,71.49771,-38.259962,84.271602,NaN,NaN,1.0,42496


In [335]:
#adjust for population growth

raw = raw.merge(df2020, on='cfips')
maska = raw["first_day_of_month"] >= "2023-01-01"
raw.loc[maska, "microbusiness_density"] = (
    np.round(raw.loc[maska, "ypred"] * raw.loc[maska, "mnoshitel"])
)

In [ ]:
test = raw[raw.first_day_of_month >= '2022-11-01'].copy()
test = test[['row_id', 'microbusiness_density']]

In [337]:
#stack two submissions
sub = pd.read_csv(DATA_PATH / "external/godaddy-lb-13803.csv")
for i, row in sub.iterrows():
    test.iat[i, 1] = 0.65 * test.iat[i, 1] + 0.35 * row["microbusiness_density"]

test.to_csv("submission.csv", index=False)


In [327]:
test.head(25)

,row_id,microbusiness_density
39,1001_2022-11-01,3.405280
40,1001_2022-12-01,3.423634
41,1001_2023-01-01,96372.767540
42,1001_2023-02-01,NaN
43,1001_2023-03-01,NaN
44,1001_2023-04-01,NaN
45,1001_2023-05-01,NaN
46,1001_2023-06-01,NaN
86,1003_2022-11-01,8.158672
87,1003_2022-12-01,8.154119
